# Modelos de Machine Learning

### Division en train y test

In [ ]:
import datetime 
df_train1 = df_final.loc['1994/01/01':'2003/12/31']
df_test = df_final.loc['2004/01/01':'2008/01/01']
df_train1.tail(10)

In [ ]:
df_train = df_train1.loc['1994/01/01':'2000/12/31']
df_val = df_train1.loc['2001/01/01':'2003/12/31']
df_val.tail(10)

## Entrenamiento de modelos

##### Separación de target y variables

In [ ]:
X_train = df_train.drop('APAC', axis = 1)
y_train = df_train['APAC']
X_val = df_val.drop('APAC', axis = 1)
y_val = df_val['APAC']
X_train_all = df_train1.drop('APAC', axis = 1)
y_train_all = df_train1['APAC']

### Problema de regresión

In [ ]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV, RandomizedSearchCV
split_index = [-1]*len(X_train) + [0]*len(X_val)
X = np.concatenate((X_train, X_val), axis=0)
y = np.concatenate((y_train, y_val), axis=0)
pds = PredefinedSplit(test_fold = split_index)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

np.random.seed(42)

In [ ]:
reg = Pipeline(steps = [
    ("scaler", MinMaxScaler()),
    ("reg", LinearRegression())])
ridge = Pipeline(steps = [
    ("scaler", MinMaxScaler()),
    ("ridge", Ridge())])
lasso = Pipeline(steps = [
    ("scaler", MinMaxScaler()),
    ("Lasso", Lasso())])
svr = Pipeline([
    ("scaler", MinMaxScaler()),
    ("selectkbest", SelectKBest()),
    ("svr", SVR())])

rand_forest = RandomForestRegressor()
xgb_reg = XGBRegressor(random_state=42)

knn_scal = Pipeline([
    ("scaler", MinMaxScaler()),
    ("knn", KNeighborsRegressor())
])

reg_param = {}

ridge_param = {'ridge__alpha': np.logspace(-3,3,10)}

lasso_param = {'Lasso__alpha': np.logspace(-3,3,10)}

svr_param = {
    "selectkbest__k": [1,2,3,4,5],
    "svr__C": np.arange(1, 100, 10),
    "svr__epsilon": np.arange(0.001, 1),
    "svr__kernel": ['linear', 'rbf', 'poly']
    }

rand_forest_param = {
    'n_estimators': [1350, 1375, 1400, 1425],
    'max_features': [4,5,6,7]
}

xgboost_params = {
        'min_child_weight': [8, 10, 12],
        'gamma': [1.5, 2, 2.5, 3, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9],
        'max_depth': [2, 3, 4]
        }

knn_param_scal = {
    'knn__n_neighbors': [3,5,7]
}



gs_reg = GridSearchCV(reg,
reg_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1)
gs_ridge = GridSearchCV(ridge,
ridge_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1)
gs_lasso = GridSearchCV(lasso,
lasso_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1)
gs_svr = RandomizedSearchCV(svr,
svr_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1, 
random_state = 42)
gs_rand_forest = GridSearchCV(rand_forest,
rand_forest_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1)

gs_xgb =  GridSearchCV(xgb_reg,
xgboost_params,
scoring='neg_mean_absolute_error',
n_jobs=4,
cv=pds,
verbose=3) 

gs_knn_scal = GridSearchCV(knn_scal,
knn_param_scal,
scoring='neg_mean_absolute_error',
cv=pds,
verbose=1,
n_jobs=-1)



grids = {
    "gs_reg": gs_reg,
    "gs_ridge": gs_ridge,
    "gs_lasso": gs_lasso,
    "gs_svr": gs_svr,
    "gs_rand_forest": gs_rand_forest,
    "gs_xgb": gs_xgb,
    "gs_knn_scal": gs_knn_scal
}

In [ ]:
%%time 
for nombre, grid_search in grids.items():
    grid_search.fit(X, y)

In [ ]:
mae_validation_dummy = np.mean(np.abs(y_val-y_train.mean()))

In [ ]:
best_grids = [(i, j.best_score_) for i, j in grids.items()]

best_grids = pd.DataFrame(best_grids, columns = ["Grid", "Best score"])
best_grids['Raes'] = (best_grids['Best score'] * (-1))/mae_validation_dummy
best_grids.sort_values(by = "Best score", ascending = False)

El mejor modelo corresponde con random_forest, que dispone de uner error relativo absoluto menor.

In [ ]:
print("Best estimator:", gs_rand_forest.best_estimator_)
print("Best params:", gs_rand_forest.best_params_)
print("Best score:", gs_rand_forest.best_score_)

### Time Series

In [ ]:
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller

from pmdarima.arima import auto_arima
from pmdarima.arima import ARIMA

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
model = auto_arima(y_train,
                   start_p = 1,
                   start_q = 1,
                   max_p = 5,
                   max_q = 5,
                   max_d = 3,
                   trace = True)

In [ ]:
print(model.aic())
predictions = model.predict(y_val.shape[0])
arima_mae = mean_absolute_error(y_val, predictions)
print("mean_absolute_error:", mean_absolute_error(y_val, predictions))
print("relative_absolute_error:", arima_mae/mae_validation_dummy)

In [ ]:
time_Serie = df_final[['APAC']].reset_index()
time_Serie['Date'] = pd.to_datetime(time_Serie['Date'])
time_Serie = time_Serie.set_index('Date')
time_Serie

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result_add = seasonal_decompose(time_Serie, model = "additive", extrapolate_trend = "freq")

plt.rcParams.update({'figure.figsize': (6,6)})
result_add.plot();

In [ ]:
for i in range(12,0,-1):
    time_Serie["t-"+str(i)] = time_Serie["APAC"].shift(i)


time_Serie.dropna(inplace=True)
time_Serie.head(15)

In [ ]:
X = time_Serie.iloc[:,1:]
y = time_Serie.iloc[:, 0]

X_train1 = X.loc['1994/01/01':'2003/12/31']
y_train1 = y.loc['1994/01/01':'2003/12/31']
X_test = X.loc['2001/01/01':'2003/12/31']
y_test = y.loc['2001/01/01':'2003/12/31']

X_train = X.loc['1994/01/01':'2000/12/31']
y_train = y.loc['1994/01/01':'2000/12/31']

X_val = X.loc['2001/01/01':'2003/12/31']
y_val = y.loc['2001/01/01':'2003/12/31']


print("Shape X_train:", X_train.shape)
print("Shape X_val", X_val.shape)
print("Shape y_train:", y_train.shape)
print("Shape y_val:", y_val.shape)

In [ ]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV, RandomizedSearchCV
split_index = [-1]*len(X_train) + [0]*len(X_val)
X = np.concatenate((X_train, X_val), axis=0)
y = np.concatenate((y_train, y_val), axis=0)
pds = PredefinedSplit(test_fold = split_index)

In [ ]:
reg = Pipeline(steps = [
    ("scaler", MinMaxScaler()),
    ("reg", LinearRegression())])
ridge = Pipeline(steps = [
    ("scaler", MinMaxScaler()),
    ("ridge", Ridge())])
lasso = Pipeline(steps = [
    ("scaler", MinMaxScaler()),
    ("Lasso", Lasso())])
svr = Pipeline([
    ("scaler", MinMaxScaler()),
    ("selectkbest", SelectKBest()),
    ("svr", SVR())])

rand_forest = RandomForestRegressor()
xgb_reg = XGBRegressor(random_state=42)

knn_scal = Pipeline([
    ("scaler", MinMaxScaler()),
    ("knn", KNeighborsRegressor())
])

reg_param = {}

ridge_param = {'ridge__alpha': np.logspace(-3,3,10)}

lasso_param = {'Lasso__alpha': np.logspace(-3,3,10)}

svr_param = {
    "selectkbest__k": [1,2,3,4,5,6],
    "svr__C": np.arange(1, 100, 10),
    "svr__epsilon": np.arange(0.001, 1),
    "svr__kernel": ['linear', 'rbf', 'poly']
    }

rand_forest_param = {
    'n_estimators': [1425, 1450, 1475, 1500],
    'max_features': [2,3,4,5,6]
}

xgboost_params = {
        'min_child_weight': [8, 10, 12],
        'gamma': [1.5, 2, 2.5, 3, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9],
        'max_depth': [2, 3, 4]
        }

knn_param_scal = {
    'knn__n_neighbors': [3,5,7]
}



gs_reg_ts = GridSearchCV(reg,
reg_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1)
gs_ridge_ts = GridSearchCV(ridge,
ridge_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1)
gs_lasso_ts = GridSearchCV(lasso,
lasso_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1)
gs_svr_ts = RandomizedSearchCV(svr,
svr_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1, 
random_state = 42)
gs_rand_forest_ts = GridSearchCV(rand_forest,
rand_forest_param,
cv=pds,
scoring = 'neg_mean_absolute_error',
n_jobs = -1,
verbose = 1)

gs_xgb_ts =  GridSearchCV(xgb_reg,
xgboost_params,
scoring='neg_mean_absolute_error',
n_jobs=4,
cv=pds,
verbose=3) 

gs_knn_scal_ts = GridSearchCV(knn_scal,
knn_param_scal,
scoring='neg_mean_absolute_error',
cv=pds,
verbose=1,
n_jobs=-1)



grids_ts = {
    "gs_reg_ts": gs_reg_ts,
    "gs_ridge_ts": gs_ridge_ts,
    "gs_lasso_ts": gs_lasso_ts,
    "gs_svr_ts": gs_svr_ts,
    "gs_rand_forest_ts": gs_rand_forest_ts,
    "gs_xgb_ts": gs_xgb_ts,
    "gs_knn_scal_ts": gs_knn_scal_ts
}

In [ ]:
%%time 
for nombre, grid_search in grids_ts.items():
    grid_search.fit(X, y)
    print(nombre)

In [ ]:
best_grids = [(i, j.best_score_) for i, j in grids_ts.items()]

best_grids = pd.DataFrame(best_grids, columns = ["Grid", "Best score"])
best_grids['Raes'] = (best_grids['Best score'] * (-1))/mae_validation_dummy
best_grids.sort_values(by = "Best score", ascending = False)

En ningún caso nigún modelo de time series mejora al modelo de random forest de regresión en el conjunto de validación.

Por lo que a continuación se entrenara este modelo con todo el conjunto de train (train + validacion) y se estudiará su actuación en test

## Modelo final y predicciones

In [ ]:
X_test = df_test.drop('APAC', axis = 1)
y_test = df_test['APAC']

In [ ]:
model = RandomForestRegressor(n_estimators= 1400, max_features=5, criterion= 'absolute_error')

model.fit(X_train_all, y_train_all)

In [ ]:
mae_validation_dummy_end = np.mean(np.abs(y_test-y_train_all.mean()))

In [ ]:
y_pred = model.predict(X_test)
mae_end = mean_absolute_error(y_test, y_pred)
rae_end = mae_end/mae_validation_dummy_end
print(mae_end)
print(rae_end)

In [ ]:
fin = pd.DataFrame(y_test)
fin['pred'] = y_pred
fin2 = fin.reset_index()
fin2

In [ ]:
# Plot the actual values
plt.plot(fin2['Date'], fin2['APAC'], 'b-', label = 'actual')
# Plot the predicted values
plt.plot(fin2['Date'], fin2['pred'], 'ro', label = 'prediction')
plt.xticks(); 
plt.legend()
# Graph labels
plt.xlabel('Date'); plt.ylabel('daily incoming solar energy in (J m-2)'); plt.title('Actual and Predicted Values');

El modelo dispone de un error relativo absoluto de 0.32, es decir el modelo mejora en un 68% al modelo trivial, predecir todo con la media del entrenamiento.